## 评价函数
本赛题采用F1-score进行评价。

对参赛者提供的结果文件（5000条记录），在A榜测试时选择40%（2000条）作为测试语料。使用B榜测试时，使用剩下的60%（3000条）。

1.   在本次评测中，我们按照“视角-情感对”作为结果评测的最小粒度。
2.   对于评测集合，逐条将参赛队伍结果中的“视角-情感对”与标注数据对比。

a)   正确数量：tp = tp1 + tp2 + tp3。

b)   情感判断错误数量：fp = fp1 + fp2 + fp3 + fp4 + fp5 + fp6

c)   漏判或多判数量：fn = fn1+fn2;
```
 	   正（标注）	中（标注）	负（标注）
正（预测）	tp1	    fp1	       fp2
中（预测）	fp3	    tp2	       fp4
负（预测）	fp5	    fp6	       tp3
视角未找到（漏判）	fn1
无效视角（多判）	 fn2
```
3.   按以下公式计算参赛者所有正确结果数量与标准结果数量的准确率(公式1)和召回率（公式2），并根据准确率和召回率计算F1-score（公式3），最后，按照F1-score对所有参赛者进行排序。
$$P = tp/(tp + fp + fn2) ----- (1)$$
        
其中，P为准确率，tp为选手提交结果中“视角-情感对”判断正确的数量，fp为选手提交结果中“视角-情感对”情感判断错误的数量。
$$R = tp/(tp + fn1) ------- (2)$$        
其中，R为召回率，tp为选手提交结果中“视角-情感对”判断正确的数量，fn为漏判和多判的数量。
$$F1 = 2 * P * R / (P+R) ------ (3)$$       
其中，F1为F1-score，P为准确率，R为召回率。

In [1]:
import pandas as pd

In [2]:
def evaluate(result_path, standard_path):
    """结果评价函数。
    @input:
        result_path: 结果 csv 文件。包括三列 [SentenceId,View,Opinion]
        standard_path: 答案 csv 文件。形式与result_path 相同。
    @return:
        P: precision
        R: recall rate
        F1: F1 value
    """
    df_result = pd.read_csv(result_path, sep='\t')
    df_standard = pd.read_csv(standard_path, sep='\t')
    df_result['id'] = df_result.SentenceId.apply(lambda num_id: str(num_id))
    df_result['key'] = df_result.id + df_result.View  # 将id和视角拼起来作为唯一标识
    df_standard['id'] = df_standard.SentenceId.apply(lambda num_id: str(num_id))
    df_standard['key'] = df_standard.id + df_standard.View  # 将id和视角拼起来作为唯一标识
    # 漏判 fn1, 多判 fn2
    r_key = set(df_result.key.values)
    s_key = set(df_standard.key.values)
    tn = len(r_key & s_key)  # 视角判断正确,取两者交集
    fn1 = len(s_key) - tn
    fn2 = len(r_key) - tn
    p_view = float(tn) / len(r_key)  # 视角提取正确率
    r_view = float(tn) / len(s_key)  # 视角提取召回率
    f1_view = 2 * p_view * r_view / (p_view + r_view) # 视角提取 F1 值
    print 'p_view=%g, r_view=%g, f1_view=%g' % (p_view, r_view, f1_view)
    # 情感判断正确 tp, 判断错误 fp
    df_result['view_opinion'] = df_result.key + df_result.Opinion  # id_view_opinion
    df_standard['view_opinion'] = df_standard.key + df_standard.Opinion  # id_view_opinion
    r_vo = set(df_result.view_opinion.values)
    s_vo = set(df_standard.view_opinion.values)
    tp = len(r_vo & s_vo)
    fp = tn - tp
    P = float(tp) / (tn + fn2)  # 准确率
    R = float(tp) / (tp + fn1)  # 召回率
    F1 = 2*P*R / (P+R)
    # print 'tn=%d, fn1=%d, fn2=%d, tp=%d, fp=%d' % (tn, fn1, fn2, tp, fp)
    print 'P=%g, R=%g, F1=%g' % (P, R, F1)

# **Example
# if __name__ == "__main__":
#     sp = "../raw_data/Label.csv"
#     rp = "../dara/result.csv"
#     evaluate(rp, sp)